
Przez to że algorytm sam w sobie jest stochastyczny nie mamy gwarancji że w trakcie jednego wywołania otrzymamy najlepszy możliwy wynik. Jedym z rozwiązać tego problemu jest zrównoleglenie algorytmu. Oznacza to, że jednocześnie śledzimy wiele "trajektorii" i na końcu obliczeń możemy wybrać najlepszą z nich. Można tego dokonać wywołując algorytm wielokrotnie, ale jest to nieefektywne. Lepszym rozwiązaniem jest trzymanie wielu stanów ułożonych w macierz. Niech $\sigma_i$ będzie stanem, wtedy mając $M$ trajektorii:

$$
\sigma = \begin{bmatrix}
\vert & \vert & \cdots & \vert \\
\sigma_1 & \sigma_2 & \cdots & \sigma_M \\
\vert & \vert & \cdots & \vert
\end{bmatrix}
$$

Wybór czy trzymamy stany w kolumnach czy w wierszach jest poniekąt arbitralny. To podejście pozwala nam równolegle liczyć wiele rozwiązań.

In [ ]:
# Implementacja zrównoleglona, z wieloma trajektoriami
import numpy as np
from math import exp
from funkcje_pomocnicze import calculate_energy_matrix
from copy import deepcopy


def random_neightbour_parrarel(solution: np.ndarray):
    rows, cols = solution.shape
    random_idx = np.random.randint(0, rows, size=cols)
    new_solution = deepcopy(solution)
    new_solution[random_idx, np.arange(cols)] *= -1
    return new_solution, random_idx


def acceptance_probability_parrarel(delta_e: np.ndarray, temp: float) -> np.ndarray:
    beta = 1 / temp
    probabilities = np.where(delta_e < 0, 1, np.exp(-beta * delta_e))
    return probabilities

def simulated_annealing_multiple_trajectories(J, h,  num_steps: int = 1000, initial_temp: float = 10, final_temp: float = 1e-12,
                                                num_repeats: int = 1, num_trajectories: int = 10,
                                              schedule: str = "linear"):
    n = len(h)
    cols = np.arange(num_trajectories)
    solution = np.random.choice([-1, 1], size=(n, num_trajectories))
    energy = calculate_energy_matrix(J, h, solution)

    T_0 = initial_temp
    T_final = final_temp

    if schedule == "linear":
        schedule = np.linspace(T_0, T_final, num=num_steps, endpoint=True)

    elif schedule == "geometric":
        alpha = 0.95
        schedule = np.array([max(T_0 * (alpha ** i), T_final) for i in range(num_steps)])

    elif schedule == "exponential":
        schedule = np.geomspace(T_0, T_final, num=num_steps, endpoint=True)

    for k in range(num_steps):
        temp = schedule[k]
        for _ in range(num_repeats):
    
            new_solution, idx = random_neightbour_parrarel(solution)
            new_energy = calculate_energy_matrix(J, h, new_solution)
            delta_e = new_energy - energy

            acc_prob = acceptance_probability_parrarel(delta_e, temp)
            r = np.random.random(size=num_trajectories)
            result = acc_prob > r

            accepted_moves = idx[result]
            changed_states = cols[result]
            solution[accepted_moves, changed_states] *=-1
            energy = calculate_energy_matrix(J, h, solution)

    return solution, energy

In [ ]:

import matplotlib.pyplot as plt
from funkcje_pomocnicze import read_instance, small_grid, test_pegasus
from tqdm import tqdm

instance = test_pegasus

J, h = read_instance(instance.path)

# Ponieważ jest to probabilistyczny algorytm warto go puścić kilka razy i wybrać najlepszy wynik

states, energies = simulated_annealing_multiple_trajectories(J, h, initial_temp=10, final_temp=1e-6, num_steps=1000, num_repeats=10, schedule="exponential")


print(f"Otrzymana energia: {min(energies)}")
print(f"Najniższa znana energia: {instance.best_energy}")
print(f"Luka energetyczna: {((instance.best_energy - min(energies))/instance.best_energy * 100):.2f}%")